In [76]:
from datetime import datetime

# SecureString parameter stored in AWS System Manager that holds a GitHub personal access token.
parameter_name = 'github_token'

# Org name to filter members
org_name = 'aws-amplify'

# Repositories for which reports will be generated.
repo_names = [
     'aws-amplify/amplify-js'
]

# Labels used across different repositories
issue_buckets = {
    'Bug': ['Bug', 'bug'],
    'Feature Request': ['Improvement', 'Feature Request', 'enhancement', 'feature-request'],
    'Question': ['Usage Question', 'Question', 'question'],
    'Pending Triage': ['Pending Triage', 'to-be-reproduced']
}

# Labels to omit from rows
omit_labels = {
    'Wont Fix', "Won't Fix", 'closing-soon-if-no-response', 'Product Review',
    'Requesting Feedback', 'Closing Soon', 'Clarification Needed', 'Duplicate',
    'Needs Info from Requester', 'work-in-progress', 'good-first-issue',
    'pending-close-response-required', 'pending-response', 'pending-release',
    'Awaiting Release', 'Investigating', 'Pending', 'Pull Request', 'duplicate',
    'good first issue', 'Reviewing', 'needs review', 'needs discussion',
    'needs-review', 'investigating', 'help wanted', 'needs-discussion',
    'Help Wanted', 'Good First Issue', 'wontfix'
}
omit_labels = omit_labels.union({label for labels in issue_buckets.values() for label in labels})

# Capture start time
start = datetime.now()

In [77]:
# Retrieve the GitHub token from SSM to prevent oops-I-pushed-credentials-to-GitHub uh-ohs.

import boto3

ssm = boto3.client('ssm')
response = ssm.get_parameter(Name=parameter_name, WithDecryption=True)
token = response['Parameter']['Value']

In [78]:
# Grab all issues from the GitHub V4 GraphQL API

import requests
import json

headers = {'Authorization': f'Bearer {token}'}


def get_issues(owner, name, *, end_cursor=None):
    query = """
    query($owner: String!, $name: String!, $endCursor: String) {
      repository(owner:$owner, name:$name) {
        issues(first:50, after:$endCursor) {
          nodes {
            number
            title
            createdAt
            closedAt
            author {
              login
            }
            reactions {
                totalCount
            }
            reactionGroups {
              users(first:50, after:$endCursor) {
                  nodes {
                      login
                  }
              }
            }
            comments(first:100, after:$endCursor) {
              nodes {
                createdAt
                author {
                  login
                }
                reactions {
                  totalCount
                }
                reactionGroups {
                    users(first:50, after:$endCursor) {
                        nodes {
                          login
                        }
                    }
                }
              }
            }
            labels(first:10) {
              nodes {
                name
              }
            }
          }
          pageInfo {
            hasNextPage
            endCursor
          }
        }
      }
    }"""
    
    variables = {'owner': owner, 'name': name, 'endCursor': end_cursor}
    response = execute(query, variables)
    page_info = response['data']['repository']['issues']['pageInfo']
    nodes = response['data']['repository']['issues']['nodes']
    
    for issue in nodes:
        yield issue
    
    if page_info['hasNextPage']:
        yield from get_issues(owner, name, end_cursor=page_info['endCursor'])
        
        
def get_members(login, *, end_cursor=None):
    query = """
    query($login: String!, $endCursor: String) {
      organization(login: $login) {
        membersWithRole(first:100, after:$endCursor) {
          nodes {
            login
          }
          pageInfo {
            hasNextPage
            endCursor
          }
        }
      }
    }"""
    
    variables = {'login': login, 'endCursor': end_cursor}
    response = execute(query, variables)
    page_info = response['data']['organization']['membersWithRole']['pageInfo']
    nodes = response['data']['organization']['membersWithRole']['nodes']
    
    for member in nodes:
        yield member['login']
    
    if page_info['hasNextPage']:
        yield from get_members(login, end_cursor=page_info['endCursor'])
        
    
def execute(query, variables):
    request = requests.post(
        'https://api.github.com/graphql',
        json={'query': query, 'variables': variables},
        headers=headers
    )

    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("{}: {}".format(request.status_code, query))

In [83]:
# Loop through each repository, grab all issues, and create a DataFrame for each.

import pandas as pd

repos = {}
bucket_lookup = {label: bucket for bucket, labels in issue_buckets.items() for label in labels}
staff = set(get_members(org_name))

for repo_name in repo_names:
    issues = []
    owner, name = repo_name.split('/')

    for issue in get_issues(owner, name):
        labels = pd.array([label['name'] for label in issue['labels']['nodes']])
        bucket = 'Other'
        created_at = pd.Timestamp(issue['createdAt'],tz='UTC')

        score = 1
        score += len(issue['comments']['nodes'])
        score += issue['reactions']['totalCount']
        score += sum(comment['reactions']['totalCount'] for comment in issue['comments']['nodes'])
        
        if issue['author']:
            impacted = {issue['author']['login']}
        else:
            impacted = set()
            
        impacted |= {comment['author']['login'] for comment in issue['comments']['nodes']
                                                if comment['author']}
        impacted |= {user['login'] for group in issue['reactionGroups'] for user in group['users']['nodes']}
        impacted |= {user['login'] for comment in issue['comments']['nodes']
                                   for group in comment['reactionGroups']
                                   for user in group['users']['nodes']}
        
        impact = len(impacted - staff)
        
        if issue['closedAt']:
            closed_at = pd.Timestamp(issue['closedAt'], tz='UTC')
        else:
            closed_at = None
            
        for label in labels:
            if label in bucket_lookup:
                bucket = bucket_lookup[label]
                break
                
        issues.append(
            [issue['number'], issue['title'], labels, created_at, closed_at, bucket, score, impact]
        )
        
    repos[repo_name] = pd.DataFrame(issues, columns=[
        'id', 'title', 'labels', 'created_at', 'closed_at', 'bucket', 'score', 'impact'
    ])

In [84]:
# Top Open Bugs by Activity

from IPython.core.display import HTML

pd.set_option('display.max_colwidth', None)

display(HTML(f'<h1>Top Open Bugs by Impact</h1>'))

for repo_name, df in repos.items():
    df = df.loc[df.bucket == 'Bug']
    df = df[~(df.closed_at > '1970-01-01')]
    df = df.sort_values(axis=0, by=['impact'], ascending=False)

    display(HTML(f'<h2>{repo_name}</h2>'))
    display(df[['id', 'title', 'created_at', 'labels', 'score', 'impact']].head(50))

,id,title,created_at,labels,score,impact
2675,4257,GraphQLResult and Observable<object> incorrect types for API.graphql,2019-10-25 04:09:48+00:00,"[GraphQL, bug]",62,44
2591,4089,support RN-0.60.+ for @aws-amplify/pushnotification,2019-09-26 16:34:03+00:00,"[Push Notifications, React Native, bug, documentation]",67,35
3701,6108,DataStore with @auth - Sync error subscription failed ... Missing field argument owner,2019-12-20 08:37:36+00:00,"[DataStore, bug]",94,30
3452,5623,Custom auth lambda trigger is not configured for the user pool.,2020-04-29 20:45:07+00:00,"[Amplify UI Components, Auth, bug]",29,19
2672,4244,React-native Auth.federatedSignIn() redirection URL mismatch error,2019-10-24 12:53:29+00:00,"[Auth, React Native, bug, documentation]",39,19
3812,6327,Custom auth lambda trigger is not configured for the user pool -> on Sign Up Confirmation,2020-07-14 23:55:10+00:00,"[Amplify UI Components, Auth, bug]",41,17
2737,4346,Amplify auth signIn throws Cors error on Microsoft Edge,2019-11-05 16:36:10+00:00,"[Browser Compatibly, Cognito, bug]",40,14
2186,3431,Delete linked user from User pool users.,2019-06-11 18:24:57+00:00,"[Cognito, Service Team, bug]",21,10
2981,4754,"implicitly has an 'any' type (zen-observable, graphql/language/ast)",2020-01-22 08:51:08+00:00,"[TypeScript, Vue, bug]",13,9
2648,4197,Amplify currentUserinfo returns null,2019-10-17 03:45:24+00:00,"[Cognito, Service Team, bug]",54,9


In [85]:
# Top Open Feature Requests by Activity

display(HTML(f'<h1>Top Open Feature Requests by Impact</h1>'))

for repo_name, df in repos.items():
    df = df.loc[df.bucket == 'Feature Request']
    df = df[~(df.closed_at > '1970-01-01')]
    df = df.sort_values(axis=0, by=['impact'], ascending=False)
    
    display(HTML(f'<h2>{repo_name}</h2>'))
    display(df[['id', 'title', 'created_at', 'labels', 'score', 'impact']].head(50))

,id,title,created_at,labels,score,impact
38,54,How to find the bidirectional map between Cognito identity ID and Cognito user information?,2017-12-10 23:17:19+00:00,"[Auth, Cognito, Service Team, feature-request]",463,168
2188,3435,cognito.user.signOut() does not invalidate tokens,2019-06-12 09:36:41+00:00,"[Cognito, Service Team, feature-request]",404,168
1003,1613,"AppSync, AWS Amplify and SSR",2018-09-11 15:30:13+00:00,"[AppSync, SSR, feature-request]",320,146
491,825,Possible to set current credentials manually?,2018-05-11 14:52:19+00:00,"[Auth, feature-request, needs-review]",143,79
185,329,Support for multiple buckets?,2018-02-21 18:49:36+00:00,"[Storage, feature-request]",119,78
741,1218,SPA with Implicit Grant - handling token refresh,2018-07-12 06:18:14+00:00,"[Auth, Service Team, Vue, feature-request]",91,57
394,661,Feature Request: Support for multiple Auth user pools,2018-04-15 23:55:42+00:00,"[Auth, feature-request]",87,56
3191,5119,DataStore: support for multi-tenant apps with sharing?,2020-03-17 12:40:28+00:00,"[DataStore, feature-request]",190,56
629,1035,Nativescript aws amplify,2018-06-15 18:04:38+00:00,[feature-request],109,55
650,1067,How to verify if a user with a given email already exists in User Pool?,2018-06-20 17:58:26+00:00,"[Cognito, feature-request]",74,54


In [86]:
# Closed Issue Report

from IPython.core.display import HTML
import pytz

pd.set_option('display.max_colwidth', None)

display(HTML(f'<h1>Closed Issues by Impact</h1>'))

thirty_days_ago = datetime.now() - pd.Timedelta('30 days')
thirty_days_ago = seven_days_ago.astimezone(pytz.timezone('UTC'))

for repo_name, df in repos.items():
    df = df[(df.closed_at > thirty_days_ago)]
    df = df.sort_values(axis=0, by=['impact'], ascending=False)

    display(HTML(f'<h2>{repo_name}</h2>'))
    display(df[['id', 'title', 'created_at', 'closed_at', 'labels', 'score', 'impact']])
    

,id,title,created_at,closed_at,labels,score,impact
3698,6104,[RFC]: Amplify Storage UI Component Refactor,2020-06-17 09:33:35+00:00,2020-09-14 21:53:39+00:00,"[Amplify UI Components, Storage, feature-request]",21,7
3352,5403,No notification when websocket closes,2020-04-14 10:11:04+00:00,2020-09-10 16:24:21+00:00,"[PubSub, bug]",15,5
3875,6469,Authenticator shows additional social button,2020-08-01 14:51:09+00:00,2020-09-11 22:31:17+00:00,"[Amplify UI Components, investigating, pending-close-response-required]",6,3
3974,6685,@aws-sdk v1.0.0-gamma.4 does not export ESM,2020-08-29 08:41:25+00:00,2020-09-10 09:23:40+00:00,"[Build, Core]",15,3
3871,6462,Credentials._setCredentialsFromSession generates invalid provider name,2020-07-31 10:33:58+00:00,2020-09-11 02:31:21+00:00,"[Auth, investigating, pending-close-response-required]",11,2
3498,5738,AWSPinPointProvider: Security Token Expires after some inactivity,2020-05-10 18:55:47+00:00,2020-09-12 20:33:18+00:00,[pending-close-response-required],24,2
4012,6755,Auth.updateUserAttribute sends a verification code,2020-09-09 03:05:37+00:00,2020-09-11 19:07:11+00:00,"[Auth, question, to-be-reproduced]",1,1
4024,6775,The conditional request fails when updating sort key of the primary index,2020-09-11 07:31:52+00:00,2020-09-14 16:58:42+00:00,"[API, GraphQL, to-be-reproduced]",1,1
4022,6770,Federation requires either a User Pool or Identity Pool in config,2020-09-10 16:32:28+00:00,2020-09-11 09:44:53+00:00,"[Auth, OAuth]",1,1
4021,6769,"packages too large, how can it be reduced?",2020-09-10 16:35:43+00:00,2020-09-10 19:43:53+00:00,[question],1,1


In [ ]:
from datetime import datetime
from IPython.display import display, HTML

total = (datetime.now() - start).seconds
minutes, seconds = divmod(total, 60)

if minutes:
    display(HTML(f'<em>Report generation took {minutes}min, {seconds}sec'))
else:
    display(HTML(f'<em>Report generation took {seconds}sec'))